## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-08-20-53-30 +0000,nypost,Owner of Trump-themed Texas burger joint faces...,https://nypost.com/2025/08/08/us-news/owner-of...
1,2025-08-08-20-52-04 +0000,nyt,Mexico’s President Says U.S. Forces Are Unwelc...,https://www.nytimes.com/2025/08/08/world/ameri...
2,2025-08-08-20-51-00 +0000,wsj,S&P 500 Posts Best Week Since June,https://www.wsj.com/finance/stocks/s-p-500-pos...
3,2025-08-08-20-49-46 +0000,nypost,Trump confirms ‘we’re going to have a meeting ...,https://nypost.com/2025/08/08/us-news/trump-co...
4,2025-08-08-20-48-32 +0000,nypost,"Slain NYC boy recently fell in with bad crowd,...",https://nypost.com/2025/08/08/us-news/slain-ny...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2311/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
1,trump,70
216,gaza,28
214,israel,18
155,plan,16
217,city,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
116,2025-08-08-16-13-00 +0000,wsj,President Trump has directed the Pentagon to b...,https://www.wsj.com/politics/national-security...,154
57,2025-08-08-19-22-00 +0000,wsj,President Trump’s team is reviewing new conten...,https://www.wsj.com/economy/central-banking/tr...,126
41,2025-08-08-19-48-00 +0000,wsj,President Trump is removing former congressman...,https://www.wsj.com/politics/policy/billy-long...,116
120,2025-08-08-16-06-14 +0000,nypost,Trump secretly authorized military force agai...,https://nypost.com/2025/08/08/us-news/trump-se...,114
125,2025-08-08-15-42-00 +0000,wsj,Trump’s Fed nominee would add a voice inside t...,https://www.wsj.com/economy/central-banking/st...,107


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
116,154,2025-08-08-16-13-00 +0000,wsj,President Trump has directed the Pentagon to b...,https://www.wsj.com/politics/national-security...
33,94,2025-08-08-20-03-13 +0000,nypost,Inside Israel’s plan to take over Gaza City an...,https://nypost.com/2025/08/08/world-news/insid...
194,42,2025-08-08-10-00-00 +0000,latimes,Masked robber put L.A. family in boarded-up cl...,https://www.latimes.com/california/story/2025-...
105,42,2025-08-08-16-47-10 +0000,nypost,JD Vance goes fishing with top UK official Dav...,https://nypost.com/2025/08/08/us-news/jd-vance...
278,41,2025-08-07-22-56-06 +0000,nyt,California Supreme Court Requires New Review o...,https://www.nytimes.com/2025/08/07/business/en...
49,35,2025-08-08-19-41-32 +0000,nypost,Texas creep wearing only a diaper yells ‘goo g...,https://nypost.com/2025/08/08/us-news/texas-cr...
133,35,2025-08-08-14-53-06 +0000,nypost,Longtime foes Armenia and Azerbaijan to ink hi...,https://nypost.com/2025/08/08/us-news/armenia-...
125,34,2025-08-08-15-42-00 +0000,wsj,Trump’s Fed nominee would add a voice inside t...,https://www.wsj.com/economy/central-banking/st...
44,30,2025-08-08-19-45-01 +0000,startribune,Apollo 13 moon mission leader James Lovell die...,https://www.startribune.com/apollo-13-moon-mis...
4,30,2025-08-08-20-48-32 +0000,nypost,"Slain NYC boy recently fell in with bad crowd,...",https://nypost.com/2025/08/08/us-news/slain-ny...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
